In [29]:
from os import name
import math
from time import localtime
from datetime import datetime,date,time,timedelta
import plotly.graph_objects as go
import numpy as np
import polars as pl
import numpy.typing as npt
from scipy import integrate

from pathlib import Path
import pprint
CURRENT_DIR = Path.cwd()
pprint.pprint(f'__name__ = {__name__}')
pprint.pprint(__name__ == '__main__')
pprint.pprint(CURRENT_DIR)

PATH_TO_DATA_DIR : str = "/home/iori/daxue/bache_thesis/20240629_down_Futamata_to_Shinjohara/"
PATH_TO_DATA_SOURCE : str = PATH_TO_DATA_DIR +"gps_omitted_firstzeros.csv"
PATH_TO_TARGET_FILE :str = PATH_TO_DATA_DIR + "gps_distance_omitted_firstzeros.csv"

'__name__ = __main__'
True
PosixPath('/home/iori/daxue/bache-thesis-python')


In [30]:
df_raw  = pl.read_csv(PATH_TO_DATA_SOURCE)

add_localtime_datetime :pl.Expr = pl.col(
    "localTimeStamp"
    ).str.to_datetime(
        format= "%Y/%m/%d %H:%M:%S%.f"
        ).alias("localtime_datetime")
add_GPStime_datetime :pl.Expr = pl.col(
    "GPStimeStamp"
    ).str.to_datetime(
        format= "%Y/%m/%d %H:%M:%S%.f"
        ).alias("GPStime_datetime")

In [31]:
df_raw = df_raw.with_columns(add_localtime_datetime).with_columns(add_GPStime_datetime).drop(["localTimeStamp","GPStimeStamp"])

In [32]:
first_localtime : datetime = df_raw["localtime_datetime"][0]
first_GPStime : datetime = df_raw["GPStime_datetime"][0]
add_localtime_timedelta : pl.Expr = (( pl.col("localtime_datetime") - first_localtime).dt.total_milliseconds()/1000 ).alias("localtime_timedelta")
add_GPStime_timedelta : pl.Expr = (( pl.col("GPStime_datetime") - first_GPStime).dt.total_milliseconds()/1000 ).alias("GPStime_timedelta")

In [33]:
df_raw = df_raw.with_columns(add_localtime_timedelta).with_columns(add_GPStime_timedelta)

In [34]:
speed_array = np.array(df_raw.get_column("speed") ) # the column 'speed' is already m/s
time_array: np.ndarray = np.array( (df_raw.get_column("localtime_timedelta")))

In [35]:
distance_array : np.ndarray = np.array([0.0 for i in time_array]) #init
memo:int = 10
for i in range(len(time_array)):
    if i==0:
        distance_array[i] = 0
    elif i < memo:
        distance_array[i] = integrate.simpson(y = speed_array[0:i], x = time_array[0:i])
    else:
        distance_array[i] = distance_array[i - memo] + integrate.simpson(y = speed_array[i-memo:i], x = time_array[i-memo:i])

In [36]:
local_time_array:np.ndarray = np.array(df_raw["localtime_datetime"].dt.to_string("%H:%M:%S%.f"))

go.Figure(
    data = [
        go.Scatter(x=local_time_array,y=speed_array*3.6,name='speed'),
        go.Scatter(x=local_time_array,y=distance_array/1000,name='distance'),
    ]
).update_layout(
    title='run curve(km/h,km)',
    legend=dict(
        xanchor='left',
        yanchor='bottom',
        x=0.02,
        y=0.9,
        orientation='h',
        )
).show()

In [37]:
distance_series:pl.Series = pl.Series('distance/m',distance_array)

In [38]:
df_with_distance = df_raw.with_columns(distance_series)

In [39]:
df_with_distance.write_csv(PATH_TO_TARGET_FILE)

In [40]:
df_raw

latitude,longitude,altitude,horizontalAccuracy,verticalAccuracy,speed,course,localtime_datetime,GPStime_datetime,localtime_timedelta,GPStime_timedelta
f64,f64,f64,f64,f64,f64,f64,datetime[μs],datetime[μs],f64,f64
34.859243,137.819527,53.559033,4.780759,4.780759,0.0,4.780759,2023-06-29 08:29:20.169,2023-06-29 08:29:20.120,0.0,0.0
34.859241,137.819527,53.434536,4.779964,4.779964,0.0,4.779964,2023-06-29 08:29:21.180,2023-06-29 08:29:21.120,1.011,1.0
34.859246,137.819533,53.556336,4.783194,4.783194,0.0,4.783194,2023-06-29 08:29:22.172,2023-06-29 08:29:22.120,2.003,2.0
34.859253,137.81953,53.701458,4.778128,4.778128,0.0,4.778128,2023-06-29 08:29:23.178,2023-06-29 08:29:23.120,3.009,3.0
34.859249,137.819532,53.647409,4.779569,4.779569,0.0,4.779569,2023-06-29 08:29:24.176,2023-06-29 08:29:24.120,4.007,4.0
…,…,…,…,…,…,…,…,…,…,…
34.723253,137.485219,38.225413,4.732471,4.732471,0.0,4.732471,2023-06-29 09:50:18.115,2023-06-29 09:50:18.056,4857.946,4857.936
34.723253,137.485219,38.225413,4.732652,4.732652,0.0,4.732652,2023-06-29 09:50:19.159,2023-06-29 09:50:19.056,4858.99,4858.936
34.723253,137.485219,38.225413,4.732857,4.732857,0.0,4.732857,2023-06-29 09:50:20.118,2023-06-29 09:50:20.056,4859.949,4859.936


In [41]:
df_with_distance

latitude,longitude,altitude,horizontalAccuracy,verticalAccuracy,speed,course,localtime_datetime,GPStime_datetime,localtime_timedelta,GPStime_timedelta,distance/m
f64,f64,f64,f64,f64,f64,f64,datetime[μs],datetime[μs],f64,f64,f64
34.859243,137.819527,53.559033,4.780759,4.780759,0.0,4.780759,2023-06-29 08:29:20.169,2023-06-29 08:29:20.120,0.0,0.0,0.0
34.859241,137.819527,53.434536,4.779964,4.779964,0.0,4.779964,2023-06-29 08:29:21.180,2023-06-29 08:29:21.120,1.011,1.0,0.0
34.859246,137.819533,53.556336,4.783194,4.783194,0.0,4.783194,2023-06-29 08:29:22.172,2023-06-29 08:29:22.120,2.003,2.0,0.0
34.859253,137.81953,53.701458,4.778128,4.778128,0.0,4.778128,2023-06-29 08:29:23.178,2023-06-29 08:29:23.120,3.009,3.0,0.0
34.859249,137.819532,53.647409,4.779569,4.779569,0.0,4.779569,2023-06-29 08:29:24.176,2023-06-29 08:29:24.120,4.007,4.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…
34.723253,137.485219,38.225413,4.732471,4.732471,0.0,4.732471,2023-06-29 09:50:18.115,2023-06-29 09:50:18.056,4857.946,4857.936,36213.647729
34.723253,137.485219,38.225413,4.732652,4.732652,0.0,4.732652,2023-06-29 09:50:19.159,2023-06-29 09:50:19.056,4858.99,4858.936,36948.529439
34.723253,137.485219,38.225413,4.732857,4.732857,0.0,4.732857,2023-06-29 09:50:20.118,2023-06-29 09:50:20.056,4859.949,4859.936,36855.300557


In [42]:
df_with_distance["distance/m"]

distance/m
f64
0.0
0.0
0.0
0.0
0.0
…
36213.647729
36948.529439
36855.300557


In [43]:
df_with_distance["distance/m"].max()

37145.231791089405